In [1]:
import numpy as np
import glob
import re
import matplotlib.pyplot as plt



In [2]:
def calculate_surface_tension(P_xx_bar, P_yy_bar, P_zz_bar, Lz_nm):
    P_xx_Pa = P_xx_bar * 101325
    P_yy_Pa = P_yy_bar * 101325
    P_zz_Pa = P_zz_bar * 101325
    Lz_m = Lz_nm * 1e-9
    gamma = 0.5 * Lz_m * (P_zz_Pa - 0.5 * (P_xx_Pa + P_yy_Pa)) * 1000
    return gamma

test

In [3]:
calculate_surface_tension(100, 100, 110, 20)

10.1325

In [6]:

def extract_number(filename):
    match = re.search(r'nsone(\d+).ptn',filename)
    if match:
        return int(match.group(1))
    return 0
# Define patterns for both directories
pattern_ptn2 = 'ptn2/ptn2/nsone[0-9]*.ptn'
pattern_ptn3 = 'ptn2/ptn3/nsone[0-9]*.ptn'
file_paths_ptn2 = glob.glob(pattern_ptn2)
file_paths_ptn3 = glob.glob(pattern_ptn3)
file_paths_ptn2 = sorted(file_paths_ptn2, key=extract_number)
combined_file_paths = file_paths_ptn2 + file_paths_ptn3


#  Method 1: average of all into 1 array

In [36]:

all_gamma_values = []

for file_path in combined_file_paths:
    
    gamma_values = []
    with open(file_path, 'r') as file:
        for line in file:
            components = line.strip().split()
            P_xx_bar = float(components[1])  # 2nd column
            P_yy_bar = float(components[5])  # 6th column
            P_zz_bar = float(components[9])  # 10th column

            # Assuming a constant Lz_nm, adjust as necessary for your simulation
            Lz_nm = 17 

            # Calculate the surface tension for these components
            gamma = calculate_surface_tension(P_xx_bar, P_yy_bar, P_zz_bar, Lz_nm)
            gamma_values.append(gamma)
        
        all_gamma_values.append(gamma_values)

In [9]:
arr = np.concatenate(all_gamma_values[:100000000])

In [17]:
averages = []

for sublist in all_gamma_values:
    if sublist:  
        avg = np.mean(sublist) 
        averages.append(avg) 

In [18]:
running_averages = []
current_sum = 0

for i, value in enumerate(arr):
    current_sum += value
    running_average = current_sum / (i + 1)  # Average up to the current index
    running_averages.append(running_average)

# Block average

In [37]:

Lz_nm = 17
all_gamma_values = []

for file_path in combined_file_paths:
    with open(file_path, 'r') as file:
        for line in file:
            components = line.strip().split()
            if len(components) >= 10:
                try:
                    P_xx_bar = float(components[1])
                    P_yy_bar = float(components[5])
                    P_zz_bar = float(components[9])
                    gamma = calculate_surface_tension(P_xx_bar, P_yy_bar, P_zz_bar, Lz_nm)
                    all_gamma_values.append(gamma)
                except ValueError:
                    continue


gamma_array = np.array(all_gamma_values)
N_chunks = 7
chunk_size = len(gamma_array) // N_chunks

block_means = []
for i in range(N_chunks):
    start = i * chunk_size
    end = start + chunk_size if i < N_chunks - 1 else len(gamma_array)
    block = gamma_array[start:end]
    block_means.append(np.mean(block))

block_means = np.array(block_means)
mean = np.mean(block_means)
sem = np.std(block_means, ddof=1) / np.sqrt(len(block_means))

print(f"Total data points: {len(gamma_array)}")
print(f"Total blocks: {len(block_means)}")
print(f"Block-averaged mean surface tension: {mean:.3f} mN/m")
print(f"Standard error from block averaging: {sem:.3f} mN/m")


Total data points: 453486
Total blocks: 7
Block-averaged mean surface tension: 28.871 mN/m
Standard error from block averaging: 0.760 mN/m
